In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np
import datetime
import time
from itertools import zip_longest
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe 
import seaborn as sns
import googlemaps
import requests
import re
import os
from geopy.geocoders import Nominatim

In [ ]:
navegador = webdriver.Chrome()  ## iniciando o webdriver do navegador

todos_os_dados = []  ## criando lista vazia para armazenar os dados de diferentes pa´ginas

pagina = 1  ## contador de páginas sendo atribuído para 1
while True:
    #utilizei a URL com filtra de imóveis para venda.
    url = f"https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o={pagina}"  ## esse valor entre páginas serve para identificar em qual página o WebDriver está lendo e por ser um contador, a cada loop uma nova página é lida
    print(f"\nAcessando página {pagina}: {url}")
    navegador.get(url)
    
    #Verifica se ainda encontra anuncios na página (Estava parando de encontrar na página 101, porém continuando a pesquisa indefinidamente.)
    try:
        navegador.find_element(By.CSS_SELECTOR, "span.typo-title-large")
        print("Nenhum anúncio encontrado nesta página. Encerrando.")
        break
    except:
        # não encontrou a mensagem, continua a raspagem
        pass

    
    try:
        WebDriverWait(navegador, 12).until(ec.presence_of_all_elements_located((
            By.CSS_SELECTOR, 'h3.olx-text.olx-text--body-large.olx-text--block.olx-text--semibold.olx-adcard__price'
        )))  ## Adicionando comando de espera para garantir que todos os elementos da página foram devidamente carregados

        elementos_valores = navegador.find_elements(By.CSS_SELECTOR, 'h3.olx-text.olx-text--body-large.olx-text--block.olx-text--semibold.olx-adcard__price')   ## localizando os elementos de preco
        elementos_localizacoes = navegador.find_elements(By.CSS_SELECTOR, 'p.olx-adcard__location')   ## localizando os elementos de localização
        anuncios = navegador.find_elements(By.CSS_SELECTOR, "div.olx-adcard__details")    

        lista_valores = [valor.text.strip() for valor in elementos_valores]  ## gerando a lista com os valores dos imóveis de uma página
        lista_localizacoes = [loc.text.strip() for loc in elementos_localizacoes] ## gerando a lista com as localizações dos imóveis de uma página
        lista_quartos = [] ## gerando a lista com qtd de qwuartos dos imóveis de uma página
        lista_areas = [] ## gerando lista com areas
        
        for anuncio in anuncios:
            detalhes = anuncio.find_elements(By.CSS_SELECTOR, "div.olx-adcard__detail")

            area = ""
            quartos = ""

            for d in detalhes:
                info = d.get_attribute("aria-label")  # extrai direto o atributo
                if info:
                    info_lower = info.lower()
                    if "metro" in info_lower or "m²" in info_lower:
                        match = re.search(r"\d+", info) ## extrai apenas os números
                        if match:
                            area = int(match.group())                        
                    elif "quarto" in info_lower:
                        match = re.search(r"\d+", info) ## Extrai apenas os números
                        if match:
                            quartos = int(match.group())
            lista_areas.append(area)
            lista_quartos.append(quartos)

        if not lista_valores:  ## se não há valores na página, não ocorre raspagem
            print("Sem valores nesta página. Fim da raspagem.")
            break

        df_valores = pd.DataFrame(
            list(zip_longest(lista_valores, lista_localizacoes, lista_areas, lista_quartos, fillvalue="")), ## garantindo que os espaços vazios nas lista sejam preenchidos com ""
            columns=["valor", "localizacao", "m2", "quartos"])
        df_valores["data"] = datetime.date.today()  ## gerando coluna com a data de extração dos registros

        todos_os_dados.append(df_valores)  ## juntando todos os dados extraídos
        print(f"Total extraído nesta página: {len(df_valores)}")

        pagina += 1  ## a cada loop, o contador de página assume +1 e a url é alterada, fazendo com que uma nova página seja lida
        time.sleep(2) ## aguarda 2 segundos para começar o novo loop


    except TimeoutException:
        print("Nenhum anúncio encontrado ou página demorou a carregar. Encerrando.")
        break   ## se ocorrer o erro Timeouexcpetion, o loop é encerrado

df_final = pd.concat(todos_os_dados, ignore_index=True)  ## juntando todos os dados obtidos de todas as páginas lidas
print(f"\nTotal geral de valores extraídos: {len(df_final)}")
print(df_final)

navegador.quit()



Acessando página 1: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=1
Total extraído nesta página: 52

Acessando página 2: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=2
Total extraído nesta página: 50

Acessando página 3: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=3
Total extraído nesta página: 50

Acessando página 4: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=4
Total extraído nesta página: 50

Acessando página 5: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=5
Total extraído nesta página: 50

Acessando página 6: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?sf=1&o=6
Total extraído nesta página: 50

Acessando página 7: https://www.olx.com.br/imoveis/venda/estado-rj/serra-angra-dos-reis-e-regiao/pe

In [4]:
lista_areas

[109,
 133,
 87,
 300,
 430,
 146,
 40,
 5000,
 100,
 5099,
 155,
 56,
 124,
 75,
 680,
 193,
 350,
 330,
 450,
 100,
 73,
 258,
 69,
 96,
 130,
 360,
 120,
 60,
 10,
 299,
 800,
 380,
 200,
 714,
 350,
 170,
 129,
 100,
 130,
 410,
 103,
 250,
 57,
 79,
 465,
 600,
 113,
 436,
 330,
 208]

In [5]:
df_final[['cidade','bairro']] = df_final['localizacao'].str.split(", ", n=1, expand=True)
df_final = df_final.drop(columns='localizacao')

In [6]:
df_final.isnull().sum()

valor      0
m2         0
quartos    0
data       0
cidade     0
bairro     2
dtype: int64

In [7]:
df_final.dropna(subset='bairro', inplace=True)

In [8]:
index_drop = df_final.loc[df_final['m2'] == ""].index
df_final = df_final.drop(index_drop)

In [11]:
#Transforma valor em Número (intt)
# limpeza inicial
df_final["valor"] = (
    df_final["valor"]
    .str.replace("R$", "", regex=False)   # remove R$
    .str.replace(".", "", regex=False)    # remove separador de milhar
    .str.replace(",", ".", regex=False)   # troca vírgula por ponto
    .str.strip()
)

# converte para número (mantém NaN nos inválidos)
df_final["valor"] = pd.to_numeric(df_final["valor"], errors="coerce")

# converte para inteiro, removendo os NaN antes
df_final["valor"] = df_final["valor"].dropna().astype(int)

In [12]:
# inicializa o geocodificador
geolocator = Nominatim(user_agent="meu_app")

# cache (dicionário)
geocode_cache = {}

def geocode_location_osm(bairro, cidade):
    endereco = f"{bairro}, {cidade}, RJ, Brasil"
    
    # se já foi consultado antes, retorna do cache
    if endereco in geocode_cache:
        return geocode_cache[endereco]
    
    # senão, consulta no Nominatim
    try:
        location = geolocator.geocode(endereco, timeout=10)
        time.sleep(1)  # respeita o limite de 1 req/segundo
        if location:
            resultado = (location.latitude, location.longitude)
        else:
            resultado = (None, None)
    except Exception as e:
        print(f"Erro ao geocodificar {endereco}: {e}")
        resultado = (None, None)
    
    # guarda no cache
    geocode_cache[endereco] = resultado
    return resultado

# aplicar no DataFrame
df_final[['latitude', 'longitude']] = df_final.apply(
    lambda row: pd.Series(geocode_location_osm(row['bairro'], row['cidade'])),
    axis=1
)


In [13]:
df_final

,valor,m2,quartos,data,cidade,bairro,latitude,longitude
0,599000.0,98,2,2025-09-15,Petrópolis,Centro,-22.509981,-43.175489
1,1480000.0,145,2,2025-09-15,Petrópolis,Bonsucesso,-22.408611,-43.149397
2,220000.0,80,2,2025-09-15,Petrópolis,Simeria,-22.536823,-43.199277
3,650000.0,85,2,2025-09-15,Petrópolis,Nogueira,-22.422239,-43.130884
4,2250000.0,146,3,2025-09-15,Petrópolis,Araras,-22.425156,-43.221496
...,...,...,...,...,...,...,...,...
4997,3995000.0,600,3,2025-09-15,Petrópolis,Itaipava,-22.412700,-43.142100
4998,900000.0,113,3,2025-09-15,Petrópolis,Corrêas,-22.442565,-43.139706
4999,2400000.0,436,3,2025-09-15,Petrópolis,Mosela,-22.493190,-43.202826
5000,4000000.0,330,4,2025-09-15,Petrópolis,Valparaíso,-22.518578,-43.191274


In [17]:
# Salvar em .csv

# gera a data de hoje no formato YYYY-MM-DD
data_hoje = datetime.date.today().strftime("%Y-%m-%d")

# monta o nome do arquivo
nome_arquivo = f"imoveis_petropolis_{data_hoje}.csv"

df_final["valor"] = df_final["valor"].fillna(0).astype(int)

# salva o DataFrame
df_final.to_csv(nome_arquivo, index=False, sep=";", encoding="utf-8-sig")

print(f"Arquivo salvo: {nome_arquivo}")
print(os.getcwd())  # mostra o diretório onde o arquivo foi salvo

Arquivo salvo: imoveis_petropolis_2025-09-15.csv
e:\Ciencia de dados\PAE\analise_imobiliaria_petropolitana
